# 1 Introduction

Here we develop a "hybrid" recommender for the Microsoft News Dataset
[@microsoft-2020] by means of the
[LightFM](https://making.lyst.com/lightfm/docs/home.html) [@kula-2015]
python package. This hybrid recommender will utilize collaborative filtering in conjunction with user 
and item (news article) information. LightFM implements a "light" version of factorization machines, including only two-way interactions. 

# 2 Configure environment

We'll require several Python modules.

In [ ]:
!pip install lightfm

In [ ]:
#| code-summary: "Load python modules"
from math import floor, inf, log10, sqrt
from os import sched_getaffinity
from os.path import join as path_join
from random import seed as set_seed, random, sample
from scipy.stats import rankdata
from sklearn.metrics import ndcg_score
from sklearn.preprocessing import normalize
import itertools as it
import json
import lightfm
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import pandas as pd
import requests
import scipy.sparse as sp
import sys
import tempfile
import zipfile

Let's define a random seed for the sake of reproducibility.

In [ ]:
#| code-summary: "Set seed"
seed = 1
set_seed(seed)

# 3 Fetch data

MIND is available to [download](https://msnews.github.io/) from Microsoft. It comprises a small variant and a large
variant. The former is a small random sub-sample of the latter. Both
variants each comprise a training set and a validation set and the large
variant comprises in addition an unlabeled testing set. Since this
official testing set is unlabeled, it won't be of much use to us, but
we'll make do with the remaining subsets. We can retrieve the data by
calling `fetch_mind()`, which is defined below.

In [ ]:
#| code-summary: "Define function to fetch data"
def fetch_mind(
    files=[
        "MINDlarge_dev.zip",
        "MINDlarge_test.zip",
        "MINDlarge_train.zip",
        "MINDsmall_dev.zip",
        "MINDsmall_train.zip"
    ],
    root="https://mind201910small.blob.core.windows.net/release"
):
    """
    Fetch Microsoft News Dataset from Internet.  In particular, download
    compressed files and de-compress them into similarly named sub-directories
    in the current directory.
    """
    for f in files:
        src = root + "/" + f
        dstdir = f.rsplit(".", 1)[0]
        r = requests.get(src, stream=True)
        if r.status_code != requests.codes.ok:
            e = "?"
            for s, n in requests.codes:
                if r.status_code == n:
                    e = s
                    break
            raise RuntimeError(("failed to fetch file `{}'; "
                + "server's response: {} ({})").format(f, r.status_code, e))
        try:
            with tempfile.TemporaryFile() as tmpfil:
                chunk_size = 2048
                for chunk in r.iter_content(chunk_size=chunk_size):
                    tmpfil.write(chunk)
                with zipfile.ZipFile(tmpfil) as zipfil:
                    zipfil.extractall(dstdir)
        except OSError as e:
            raise RuntimeError("failed to decompress file `{}': {}".
                format(f, e))

For example, the following chunk if evaluated will attempt to fetch the
small variant's training and validation sets.

In [ ]:
#| code-summary: "Fetch data example"

fetch_mind([
    "MINDsmall_train.zip",
    "MINDsmall_dev.zip"
])

# 4 Prepare data

Now that we have a local copy of MIND (or of a subset of it), let's
process it for analysis.

## Structure of MIND

Each partition of MIND comprises:

-   behaviors data (file named "behaviors.tsv")

-   news data ("news.tsv")

-   embeddings of named entities ("entity_embedding.vec")

-   embeddings of relations of named entities
    ("relation_embedding.vec")

Detailed information about these data is available
[elsewhere](https://github.com/msnews/msnews.github.io/blob/811c3da00f028ad7737d8c8e131770e04ffe6346/assets/doc/introduction.md),
but a few details will be repeated here for the sake of this tutorial.

### Behaviors data

The behaviors dataset contains users' implicit feedback about news items in
the form of history logs and impressions logs. MIND was
assembled from click-activity that occurred over several weeks; the
history log is a list of items that a user is known to have consumed
during an initial portion this period. The impressions log is a list of
items that were displayed to a user at a particular time afterward
together with the user's implicit feedback about these items (1 = click, 0 = no click).

In [ ]:
#| code-summary: "Set name of behaviors file"
behaviors = "behaviors.tsv"

In [ ]:
#| code-summary: "Define function to read behaviors data"
def get_behaviors(path):
    """Read behaviors data from file, return as DataFrame."""
    pdargs = {
        "sep": "\t",
        "header": None,
        "names": ["impr_id", "user_id", "time", "history", "impr"],
        "dtype": {
            "impr_id": str,
            "user_id": str,
        },
        "parse_dates": ["time"],
        "converters": {
            "history": lambda x: x.split(),
            "impr":    lambda s: [(item, int(label)) for x in s.split()
                           for item, label in [x.rsplit("-", 1)]],
        },
    }
    with open(path) as f:
        pdargs["filepath_or_buffer"] = f
        ret = pd.read_csv(**pdargs)
    return ret

### News data

The news data is comprised of metadata about the news items that may be present
in the history and impressions logs of the behavior data. Fields include
category, sub-category, and named entities.

In [ ]:
#| code-summary: "Set name of news data file"
news = "news.tsv"

In [ ]:
#| code-summary: "Define function to read news data"
def get_news(path):
    """Read news data from file, return as DataFrame."""
    # pd.read_csv parses MINDsmall_train/news.tsv incorrectly, so we'll do it
    # more manually.
    cn = [
        "item", "cat", "subcat", "title", "abstract", "url", "ent_t", "ent_a"
    ]
    conv = ([lambda x: x] * 6) + ([lambda x: json.loads(x)] * 2)
    rec = []
    with open(path) as f:
        for lineno, line in enumerate(f):
            x = line.strip().split("\t")
            assert len(x) == len(conv),\
                "expected {} fields, observed {} in record {}: `{}'".\
                    format(len(cn), len(x), lineno, line)
            try: rec.append([conv[i](xi) for i, xi in enumerate(x)])
            except json.decoder.JSONDecodeError as e:
                print("failed to decode JSON in record {}: {}".
                    format(lineno, e), file=sys.stderr)
    return pd.DataFrame.from_records(rec, columns=cn)

### Entity-embeddings data

The entity-embeddings dataset contains one-hundred dimensional embeddings of
some of the named entities in the news dataset.

In [ ]:
#| code-summary: "Set name of entity embeddings file"
entity_embeddings = "entity_embedding.vec"

In [ ]:
#| code-summary: "Define function to read entity embeddings"
def get_embeddings(path):
    """Read embeddings data from file, return as DataFrame."""
    cn = ["id"] + ["x" + str(i) for i in range(100)]
    conv = [lambda x: x] + ([lambda x: float(x)] * (len(cn) - 1))
    rec = []
    with open(path) as f:
        for lineno, line in enumerate(f):
            x = line.strip().split("\t")
            assert len(x) == len(conv),\
                "expected {} fields, observed {} in record {}: `{}'".\
                    format(len(cn), len(x), lineno, line)
            try: rec.append([conv[i](xi) for i, xi in enumerate(x)])
            except ValueError as e:
                print("failed to decode number in record {}: {}".
                    format(lineno, e), file=sys.stderr)
    return pd.DataFrame.from_records(rec, columns=cn)

### Relation-embeddings data

The relation-embeddings dataset contains one-hundred dimensional embeddings of
knowledge-graph relations of some of the named entities in the news dataset. We
won't do anything with them here.

## Read data

In [ ]:
#| code-summary: "Select data sets"

dir_training = "MINDsmall_train"
dir_testing  = "MINDsmall_dev"

Let's fetch our training set from the directory `dir_training` and our
testing set from `dir_testing`. We'll allocate later a subset of the
training set for validation.

In [ ]:
#| code-summary: "Fetch data"
data = {}
dirs = {
    "tr": dir_training,
    "te": dir_testing,
}
for k, d in dirs.items():
    data[k] = {}
    data[k]["beha"] = get_behaviors(path_join(d, behaviors))
    data[k]["news"] = get_news(path_join(d, news))
    data[k]["ents"] = get_embeddings(path_join(d, entity_embeddings))

Let's glimpse the training set.

In [ ]:
#| code-summary: "Glimpse of training set"
for k, v in data["tr"].items():
    print("{}:\n\tshape: {}\n\tcolumns: {}".format(k, v.shape, v.columns))

## Partition training set

As mentioned previously, we'll partition the training set in two, reserving 
some data for validation. Our partitioning scheme will reserve
for validation the training-set data from the most recent day on which
training data was collected; the remainder will be retained for
training during hyperparameter-tuning. In addition, we'll retain the
complete training set for training a final model after having completed
the hyperparameter-tuning.

In [ ]:
#| code-summary: "Inspect date range of original training set"
print(data["tr"]["beha"]["time"].agg([min, max]))

In [ ]:
#| code-summary: "Partition the training set"

# Initialize validation data from training data
data["va"] = dict(data["tr"])

# Copy full training set for training final model after tuning
# hyper-parameters.
data["tr_full"] = dict(data["tr"])

# Reserve for validation the data associated with last day of training period.
t0 = data["tr_full"]["beha"]["time"].max() - pd.Timedelta("1 day")
p = data["tr_full"]["beha"]["time"] <= t0
data["tr"]["beha"] = data["tr_full"]["beha"].loc[p]
data["va"]["beha"] = data["tr_full"]["beha"].loc[~p]
for k in ["tr", "va"]:
    print("{}-set behaviors table comprises {} records".format(k,
        data[k]["beha"].shape[0]))

## Assemble interactions matrices

For each subset of the data (training, validation, testing) we'll
require a sparse matrix (in coordinate-list format) of user-item
interactions. The rows correspond to users, columns correspond to
items and entries specify the users' implicit feedback about the
items; something like the following for each partition of the data-set,
where ones indicate (implicit) positive feedback and zeros (implicit)
negative feedback.

            item_1 item_2 ... item_n
    user_1 [     1      0 ...      1]
    user_2 |     0      0 ...      1|
     ...   |   ...    ... ...    ...|.
    user_m [     1      1 ...      0]

In [ ]:
#| code-summary: "Create sparse matrices"
for k in data.keys():
    users, items, ratings = [], [], []
    for row in data[k]["beha"].itertuples():
        for item, rating in row.impr:
            users.append(row.user_id)
            items.append(item)
            ratings.append(rating)
    data[k]["users"] = users
    data[k]["items"] = items
    data[k]["ratings"] = ratings

The correspondence between users and rows and between items and columns
must be the same for all partitions of the data-set, so that for example
user $u$ is associated with row $r$ and item $i$ with column $c$
irrespective of whether $(u,i)$ comes from the training set, from the
validation set or from the testing set. To facilitate this, we'll create maps.

In [ ]:
#| code-summary: "Create user map"
user_map = {}
for v in data.values():
    for x in v["users"]:
        _ = user_map.setdefault(x, len(user_map))
print("{} users present in the data-set".format(len(user_map)))

In [ ]:
#| code-summary: "Create item map"
item_map = {}
for v in data.values():
    for x in v["items"]:
        _ = item_map.setdefault(x, len(item_map))
print("{} items present in the data-set".format(len(item_map)))

Having established these correspondences, we're ready to assemble the
interactions matrices from the user-item-rating triples.

In [ ]:
#| code-summary: "Assemble interaction matrices"
shape = (len(user_map), len(item_map))
for k, v in data.items():
    data[k]["ui"] = sp.coo_matrix((v["ratings"], (
        [user_map[x] for x in v["users"]],
        [item_map[x] for x in v["items"]]
    )), shape=shape)
    print(("{}-set interactions matrix has {} rows, {} columns and {} non-"
        + "zero entries").format(k, *data[k]["ui"].shape,
        data[k]["ui"].count_nonzero()))

We need not store explicit zeros in the training-set interactions matrix:
Implicit zeros will suffice for training, because LightFM does its own
sampling of implicit negatives.

In [ ]:
#| code-summary: "Eliminate zeros"
data["tr"]["ui"].eliminate_zeros()
data["tr_full"]["ui"].eliminate_zeros()

However, we do wish to retain any explicit zeros found in the validation
and testing sets to retain the identity of the specific user-item pairs
for which to make predictions during validation and during testing.

## Assemble features matrices

In addition to interactions data, LightFM can incorporate data about the
interactions into the recommendation system; we'll call these additional
data "features". In particular, we can incorporate user-features and
item-features (a.k.a. side information) by passing LightFM separate sparse matrices whose rows
correspond to users and to items respectively, whose columns correspond
to features, and whose entries specify the users' and the items' values
for these features.

### User-features

We'll start with user-features. Each partition of the data-set
(training, validation, testing) will have a user-features matrix, which
we'll organize in blocks (sub-matrices) for convenience. All blocks of a
given partition will share the mapping of users to rows established for
the interactions matrices, but each will have its own mapping of
features to columns that will be shared with the corresponding blocks of
the other partitions. After we've assembled the blocks, we'll stack them
side-by-side to form the final user-features matrix.

Here's a diagram. Suppose that our user-features matrix, `U`, comprises
`p` features, `F1, F2, ..., Fp`. Then we might have

    U = [F1 F2 ... Fp].

Note that the feature blocks `F1, F2, ..., Fp` share the same number of
rows: one per user. However, they may have different numbers of columns
according to the underlying data that they represent; for example, a
continuous feature `Fj` may be represented as a single column,

         [x] user_1
    Fj = |y|  ...   ,
         [z] user_m

whereas a categorical feature `Fk` of `g` levels will be one-hot encoded
in as many columns,

          level_1 level_2 ... level_g
         [      1       0 ...       0] user_1
    Fk = |      1       0 ...       0|  ...   .
         [      0       1 ...       0] user_m

In [ ]:
#| code-summary: "Create dictionaries"
uft_blk = {k: {} for k in data.keys()}
uft_map = {k: {} for k in data.keys()}

Specifically, in the user-features matrix we'll facilitate inclusion of
the features named in `which_user_ft`. To use LightFM for traditional 
collaborative filtering (with no side information), specify
`user` alone here, along with `item` alone later on in `which_item_ft`.

In [ ]:
#| code-summary: "Choose user features"

# Comment out any of the entries of `which_user_ft' to exclude these from the
# features matrix.
which_user_ft = [
    "user",
    "history",
]

The identity-features block is merely the identity matrix that has a one
for each user present in the data-set,

            user_1 user_2 ... user_m
    user_1 [     1     0  ...      0]
    user_2 |     0     1  ...      0|
     ...   |   ...   ...  ...    ...|
    user_m [     0     0  ...      1]

In [ ]:
#| code-summary: "Assemble identity block of user-features matrices"
blk_name = "user"
if blk_name in which_user_ft:
    blk = sp.eye(len(user_map), format="coo")
    print("block `{}' has {} rows, {} columns, {} non-zero entries".
        format(blk_name, *blk.shape, blk.count_nonzero()))
    for s in data:
        uft_blk[s][blk_name] = blk
        uft_map[s][blk_name] = user_map

The block associated with users' histories is only a little more
complex. Some of the work that we'll do for this block we'll repeat for
other features' blocks, so we'll first define a function for re-use. The
purpose of this function is to ensure that the row- and column-mappings
associated with the block under consideration are the same for all
partitions of the data-set, so that for example the training set's
history block associates the same rows and users and the same columns
and items as the validation set's history block.

In [ ]:
#| code-summary: "Define function for getting feature blocks"
def get_ft_blk(data, get_ijv, row_map):
    ijv = [get_ijv(x) for x in data]
    col_map = {}
    for j in it.chain.from_iterable(x[1] for x in ijv):
        _ = col_map.setdefault(j, len(col_map))
    blk = []
    for i, j, v in ijv:
        _i = [row_map[x] for x in i]
        _j = [col_map[x] for x in j]
        blk.append(sp.coo_matrix((v, (_i, _j)),
            shape=(len(row_map), len(col_map))))
    return (blk, col_map)

The rows of the history block will correspond to users and the columns
to the items present in the users' histories. For each user `u`, the
items present in the user's history log will receive a weight of one
divided by the total number of items in the user's log---let there be
`k_u` of them---so that each non-zero row of the block sums to one,

            item_1         item_2 ...     item_n
    user_1 [1/k_user_1          0 ... 1/k_user_1]
    user_2 |         0          0 ... 1/k_user_2|
     ...   |   ...            ... ...        ...|.
    user_m [1/k_user_m 1/k_user_m ...          0]

In [ ]:
#| code-summary: "Assemble history block of user-features matrices"
blk_name = "history"
if blk_name in which_user_ft:
    def get_ijv(data):
        I, J, V = [], [], []
        for u, udat in data.groupby("user_id"):
            xs = set(it.chain.from_iterable(udat["history"]))
            I.extend(u for x in xs)
            J.extend(xs)
            V.extend(1/len(xs) for x in xs)
        return (I, J, V)
    blks, col_map = get_ft_blk(
        data    = (x["beha"] for x in data.values()),
        get_ijv = get_ijv,
        row_map = user_map
    )
    for s, b in zip(data.keys(), blks):
        print(("{}-set block `{}' has {} rows, {} columns, {} non-"
           + "zero entries").format(s, blk_name, *b.shape, b.count_nonzero()))
        uft_blk[s][blk_name] = b
        uft_map[s][blk_name] = col_map

We can now combine the blocks of the user-features matrices.

In [ ]:
#| code-summary: "Combine blocks of the user-features"
for k in data.keys():
    data[k]["uft"] = sp.hstack([x for x in uft_blk[k].values()], format="csr")
    print(("{}-set user-features matrix has {} rows, {} columns, "
        + "{} non-zero entries").format(k, *data[k]["uft"].shape,
        data[k]["uft"].count_nonzero()))
    if data[k]["uft"].shape == (0, 0):
        data[k]["uft"].shape = None

We might want to normalize the non-zero values of the matrices (as does
`lightfm.Dataset.build_user_features`).

In [ ]:
#| code-summary: "Normalize non-zero values"
for k in data.keys():
    if data[k]["uft"] is not None:
        data[k]["uft"] = normalize(data[k]["uft"].tocsr(), norm="l1",
            copy=False)

### Item-features

Next come the item-features. Each partition of the data-set (training,
validation, testing) will have an item-features matrix, which we'll
organize in blocks like we've done with the user-features matrices. All
blocks of a given partition will share the mapping of items to rows
established between items and columns for the interactions matrices, but
each will have its own mapping of features to columns that will be
shared with the corresponding blocks of the other partitions. After
we've assembled the blocks, we'll stack them side-by-side to form the
final item-features matrix.

In [ ]:
#| code-summary: "Create dictionaries"
ift_blk = {k: {} for k in data.keys()}
ift_map = {k: {} for k in data.keys()}

Specifically, in the item-features matrix we'll facilitate inclusion of
the features named in `which_item_ft`. Note that this isn't an
exhaustive list of the item-features that one might possibly include;
consider, for example, that we could (but won't) incorporate the text of
items' titles and abstracts with some more complex pre-processing.

In [ ]:
#| code-summary: "Choose item features"

# Comment out any of the entries of `which_item_ft' to exclude these from the
# features matrix.
which_item_ft = [
    "item",
    "cat",
    "subcat",
    "label_ent_a",
    "label_ent_t",
#    "embedding_ent_a",
#    "embedding_ent_t",
]

The identity-features block is merely the identity matrix that has a one
for each item present in the data-set,

            item_1 item_2 ... item_n
    item_1 [     1     0  ...      0]
    item_2 |     0     1  ...      0|
     ...   |   ...   ...  ...    ...|
    item_n [     0     0  ...      1]

In [ ]:
#| code-summary: "Assemble identity block of item-features matrices"
blk_name = "item"
if blk_name in which_item_ft:
    blk = sp.eye(len(item_map), format="coo")
    print("block `{}' has {} rows, {} columns, {} non-zero entries".
        format(blk_name, *blk.shape, blk.count_nonzero()))
    for s in data:
        ift_blk[s][blk_name] = blk
        ift_map[s][blk_name] = item_map

Most of the item-features data resides in the news tables. These tables
may comprise items that aren't present in any of the behaviors tables,
which we may as well ignore.

In [ ]:
#| code-summary: "Delete data on irrelevant items"
for k, v in data.items():
    data[k]["news"] = v["news"].loc[
        v["news"]["item"].isin(item_map.keys())
    ]
    assert not any(data[k]["news"]["item"].duplicated()),\
        "duplicate items in {}-set news data".format(k)

The category and sub-category blocks will align items in rows (as
previously stated) and their categories and sub-categories in columns.
There's exactly one category and one sub-category associated with each
item, each of which will be given a weight of one. Here follows a
diagram of one possible such block, where we assume that there are `g`
categories.

            category_1 category_2 ... category_g
    item_1 [         1          0 ...          0]
    item_2 [         0          1 ...          0]
      ...  |       ...        ... ...        ...|
    item_n [         1          0 ...          0]

In [ ]:
#| code-summary: "Assemble category, sub-category blocks of item-features matrices"
for blk_name in ["cat", "subcat"]:
    if blk_name not in which_item_ft:
        continue
    def get_ijv(x):
        I, J, V = [], [], []
        for row in x.itertuples():
            I.append(row.item)
            J.append(getattr(row, blk_name))
            V.append(1)
        return (I, J, V)
    blks, col_map = get_ft_blk(
        data    = (x["news"] for x in data.values()),
        get_ijv = get_ijv,
        row_map = item_map
    )
    for s, b in zip(data.keys(), blks):
        print(("{}-set block `{}' has {} rows, {} columns, {} non-"
           + "zero entries").format(s, blk_name, *b.shape, b.count_nonzero()))
        ift_blk[s][blk_name] = b
        ift_map[s][blk_name] = col_map

Next, we'll assemble blocks for the named entities of items' titles and
abstracts, in particular, for the associated labels (such as "PGA Tour"
for the corresponding named entity). Note that there may be zero or more
named entities per title/abstract per item, which we'll weight equally
so that the weights of each item's named entities sum to one,

            entity_1     entity_2 ...   entity_g
    item_1 [1/k_item_1          0 ... 1/k_item_1]
    item_2 |         0          0 ...          0|
     ...   |   ...            ... ...        ...|
    item_n [1/k_item_n 1/k_item_n ...          0]

In [ ]:
#| code-summary: "Assemble blocks for named entities from item titles and abstracts"
for k in ["ent_t", "ent_a"]:
    blk_name = "label_" + k
    if blk_name not in which_item_ft:
        continue
    def get_ijv(data):
        I, J, V = [], [], []
        for row in data.itertuples():
            xs = set(x.get("Label") for x in getattr(row, k))
            xs.discard(None)
            n = len(xs)
            for x in xs:
                I.append(row.item)
                J.append(x)
                V.append(1 / n)
        return (I, J, V)
    blks, col_map = get_ft_blk(
        data    = (x["news"] for x in data.values()),
        get_ijv = get_ijv,
        row_map = item_map
    )
    for s, b in zip(data.keys(), blks):
        print(("{}-set block `{}' has {} rows, {} columns, {} non-"
           + "zero entries").format(s, blk_name, *b.shape, b.count_nonzero()))
        ift_blk[s][blk_name] = b
        ift_map[s][blk_name] = col_map

Next, we'll assemble blocks for the named entities' one-hundred
dimensional embeddings. The blocks will associate each item with at most
one embedding, although some items' named entities may lack embeddings
and some items may be associated with more than one named entity. In the
first case, the blocks' corresponding rows will be zero; in the second
case, the embeddings will first be summed; and, in any case, the blocks'
non-zero rows will be scaled to each have unit length,

            embedding_1 embedding_2 ...   embedding_100
    item_1 [          x           a ...               q]
    item_2 |          y           b ...               r|
     ...   |        ...         ... ...             ...|
    item_n [          z           c ...               s]

(In this diagram, the letters inside the matrix denote arbitrary real
numbers.)

In [ ]:
#| code-summary: "Assemble blocks for the one-hundred dimensional embeddings of named entities"
for k in ["ent_t", "ent_a"]:
    blk_name = "embedding_" + k
    if blk_name not in which_item_ft:
        continue
    def get_ijv(data):
        I, J, V = [], [], []
        news, ents = data["news"], data["ents"]
        ents_by_item = news[["item", k]].assign(
            wdid=lambda row: [
                list(filter(None, (x.get("WikidataId") for x in xs)))
                    for xs in row[k]
            ]
        ).explode("wdid").merge(
            ents, left_on="wdid", right_on="id"
        ).drop(columns=["id", "wdid", k]).groupby("item").sum().transform(
            lambda x: x / sqrt(x.dot(x)), axis=1
        )
        for ix, x in np.ndenumerate(ents_by_item.to_numpy()):
            I.append(ents_by_item.index[ix[0]])
            J.append(ents_by_item.columns[ix[1]])
            V.append(x)
        return (I, J, V)
    blks, col_map = get_ft_blk(
        data    = data.values(),
        get_ijv = get_ijv,
        row_map = item_map
    )
    for s, b in zip(data.keys(), blks):
        print(("{}-set block `{}' has {} rows, {} columns, {} non-"
           + "zero entries").format(s, blk_name, *b.shape, b.count_nonzero()))
        ift_blk[s][blk_name] = b
        ift_map[s][blk_name] = col_map

We can now combine the blocks of the item-features matrices.

In [ ]:
#| code-summary: "Combine blocks of item-features matrices"
for k in data.keys():
    data[k]["ift"] = sp.hstack([x for x in ift_blk[k].values()], format="csr")
    print(("{}-set item-features matrix has {} rows, {} columns, "
        + "{} non-zero entries").format(k, *data[k]["ift"].shape,
        data[k]["ift"].count_nonzero()))
    if data[k]["ift"].shape == (0, 0):
        data[k]["ift"].shape = None

We might want to normalize the non-zero values of the matrices (as does
`lightfm.Dataset.build_item_features`).

In [ ]:
#| code-summary: "Normalize non-zero values"
for k in data.keys():
    if data[k]["ift"] is not None:
        data[k]["ift"] = normalize(data[k]["ift"].tocsr(), norm="l1",
            copy=False)

# 5 Train model

We're ready to train a recommender by means of LightFM. There are many
hyper-parameters to specify, such as the dimension of the latent feature
space, the optimization criterion and the number of iterations of
training. For simplicity, we'll leave many unspecified, which will
therefore assume LightFM's default values. More information is available 
[here](https://making.lyst.com/lightfm/docs/lightfm.html).

In [ ]:
#| code-summary: "Set hyper-parameters"
hyper = {
    "dim":      50,
    "loss":     "bpr", # Bayesian personalized ranking
    "iter":     100,
    "rate":     0.05,
    "patience": 10,
}

In [ ]:
#| code-summary: "Define model"
model = lightfm.LightFM(
    no_components = hyper["dim"],
    loss          = hyper["loss"],
    learning_rate = hyper["rate"],
    random_state  = seed
)

We'll implement early-stopping to facilitate tuning. If this is enabled,
training will cease early after `hyper["patience"]` iterations of
training that don't improve the recommender's best validation-set
performance as per the stopping criterion `hyper["stop_crit"]` (defined
below). Note that training will likely take considerably longer if
early-stopping is enabled.

The LightFM module does not provide access to the loss function values, so
early-stopping will be based on an accuracy-based metric.

In [ ]:
#| code-summary: "Define functions used for early stopping"

# For early-stopping based on normalized discounted cumulative gain
def mean_ndcg(cond, pred):
    return mean_row_stat(cond, pred, ndcg)

# For early-stopping based on hit-rate at k
def mean_hrk(cond, pred):
    return mean_row_stat(cond, pred, hrk)

# For early-stopping based on mean reciprocal rank
def mean_recip_rank(cond, pred):
    return mean_row_stat(cond, pred, recip_rank)

# Function definition enabling collection of metrics
def mean_row_stat(a, b, f):
    """
    Return mean of `f' applied in parallel to corresponding non-zero rows
    of LIL-format matrices `a' and `b'.
    """
    with multiprocessing.Pool(n_avail_cpu()) as pool:
        pairs = ((x, y) for x, y in zip(a.data, b.data) if len(x) and len(y))
        return np.mean(pool.starmap(f, pairs))

In [ ]:
#| code-summary: "Setup for parallel processing"
def n_avail_cpu():
    """Return the number of central processing units available for use."""
    return len(sched_getaffinity(0))

In [ ]:
#| code-summary: "Define evaluation metrics"

# Normalized Discounted Cumulative Gain
def ndcg(cond, pred, k=5):
    """
    Compute normalized discounted cumulative gain at k = `k' of list
    of predictions `pred' given corresponding labels `cond'.
    """
    a = np.array(cond).reshape(1, -1)
    b = np.array(pred).reshape(1, -1)
    return ndcg_score(a, b, k=k)

# Hit rate at k
def hrk(rel_true, rel_pred, k=5):
    """
    Return hit-rate at `k', that is, the proportion of relevant items
    (`rel_true' > 0) whose predicted relevance (`rel_pred') is among the
    highest `k'.
    """
    n = len(rel_true)
    if len(rel_pred) != n:
        raise ValueError("lists of relevance-values differ in length")
    # We shuffle the data to randomize the breaking of ties in ranks.
    js = sample(range(n), k=n)
    ps = [rel_true[j] > 0 for j in js]
    qs = (r <= k for r in rankdata([-rel_pred[j] for j in js], "ordinal"))
    den = sum(ps)
    return inf if not den else sum(p & q for p, q in zip(ps, qs)) / den

# Reciprocal rank
def recip_rank(rel, pred):
    """
    Return reciprocal rank of first relevant (`rel' > 0) element, where ranks
    are awarded in order of non-increasing values of `pred' so that the lowest
    rank is assigned to the highest value, the second-lowest rank to the
    second-highest value and so on.
    """
    n = len(rel)
    if len(rel) != n:
        raise ValueError("lists of relevance-values differ in length")
    # We shuffle the data to randomize the breaking of ties in ranks.
    js = sample(range(n), k=n)
    is_rel = [rel[j] > 0 for j in js]
    ranks = rankdata([-pred[j] for j in js], "ordinal")
    for p, r in zip(is_rel, ranks):
        if p:
            return 1 / r
    return inf

In [ ]:
#| code-summary: "Run to enable early-stopping"

#hyper["stop_crit"] = mean_ndcg

If early-stopping is disabled (above) then we'll merely train a recommender with
the specified values of hyper-parameters on the full training set.
Otherwise, we'll train on the reduced training set and evaluate the
model after each epoch of training on the validation set until reaching
the stopping point defined by `hyper["iter"]` and by the early-stopping
criterion.

In [ ]:
#| code-summary: "Fit model"

%matplotlib inline

if hyper.get("stop_crit") is None:
    # Train on full training set since we aren't tuning any hyper-parameters.
    model.fit(
        interactions  = data["tr_full"]["ui"],
        epochs        = hyper["iter"],
        user_features = data["tr_full"]["uft"],
        item_features = data["tr_full"]["ift"],
        num_threads   = n_avail_cpu()
    )
else:
    stats = []
    whichmax = 0
    for i in range(hyper["iter"]):
        # Train for one epoch on one of the partitions of the training set.
        model.fit_partial(
            interactions  = data["tr"]["ui"],
            epochs        = 1,
            user_features = data["tr"]["uft"],
            item_features = data["tr"]["ift"],
            num_threads   = n_avail_cpu()
        )
        # Evaluate model on other partition.
        yhat = model.predict(
            user_ids      = data["va"]["ui"].row,
            item_ids      = data["va"]["ui"].col,
            user_features = data["va"]["uft"],
            item_features = data["va"]["ift"],
            num_threads   = n_avail_cpu()
        )
        cond = sp.coo_matrix(data["va"]["ui"])
        pred = sp.coo_matrix((yhat, (cond.row, cond.col)), shape=cond.shape)
        stat = hyper["stop_crit"](cond.tolil(), pred.tolil())
        print("validation-set statistic on iteration {:{}d}: {:f}".
            format(i, 1 + floor(log10(hyper["iter"])), stat))
        # Evaluate stopping criterion.
        stats.append(stat)
        if stat > stats[whichmax]:
            whichmax =  i
            continue
        if i - whichmax >= hyper["patience"]:
            print("stopping early...best iteration: {}".format(whichmax))
            break
    # Re-train on full training set for number of epochs adjusted according to
    # difference in size between full training set and partial training set.
    model.fit(
        interactions  = data["tr_full"]["ui"],
        epochs        = int(1 + whichmax
                            * data["tr_full"]["ui"].count_nonzero()
                            / data["tr"]["ui"].count_nonzero()),
        user_features = data["tr_full"]["uft"],
        item_features = data["tr_full"]["ift"],
        num_threads   = n_avail_cpu()
    )
    # Illustrate validation-set statistics.
    fig, ax = plt.subplots()
    ax.plot(stats, "bo")
    ax.set_xlabel("epoch")
    ax.set_ylabel(hyper["stop_crit"].__name__)
    ax.set_title("validation-set statistics by training epoch")
    plt.show()

# 6 Evaluate model

Let's evaluate our trained recommender on the testing set. We'll compute
the evaluation metrics in `metrics` (see below) from the recommender's
predictions for the user-item pairs in the testing set.

In [ ]:
#| code-summary: "Specify evaluation metrics"

metrics = {
    "mean HR@k":            mean_hrk,
    "mean NDCG":            mean_ndcg,
    "mean reciprocal rank": mean_recip_rank,
}

In [ ]:
#| code-summary: "Score testing set"

yhat = model.predict(
    user_ids      = data["te"]["ui"].row,
    item_ids      = data["te"]["ui"].col,
    user_features = data["te"]["uft"],
    item_features = data["te"]["ift"],
    num_threads   = n_avail_cpu()
)

In [ ]:
#| code-summary: "Evaluate model for all users"

# The implementations of the evaluation metrics in `metrics' each take two
# LIL-format sparse user-item matrices as input: One assumed to provide
# labels (`cond') and, the other (`pred'), predictions.
cond = sp.coo_matrix(data["te"]["ui"])
pred = sp.coo_matrix((yhat, (cond.row, cond.col)), shape=cond.shape)
# for all users
_cond = cond.tolil()
_pred = pred.tolil()
print("for all users:")
for s, f in metrics.items():
    print("\t{}: {:f}".format(s, f(_cond, _pred)))

Note that there are users in the test set, which were not in the training set 
(a "cold start" situation). Below we consider two sets of users: (1) those that appear 
in both train and test, and (2) those that appear only in the test set.

In [ ]:
#| code-summary: "Evaluate model for users present both in training and testing sets"

# for users present both in training set and in testing set
I = list(set(data["te"]["ui"].row) & set(data["tr_full"]["ui"].row))
_cond = cond.tolil()[I]
_pred = pred.tolil()[I]
print("for users present both in training set and in testing set:")
for s, f in metrics.items():
    print("\t{}: {:f}".format(s, f(_cond, _pred)))

In [ ]:
#| code-summary: "Evaluate model for testing-set users not present in training set"

# for testing-set users *not* present in training set
I = list(set(data["te"]["ui"].row) - set(data["tr_full"]["ui"].row))
_cond = cond.tolil()[I]
_pred = pred.tolil()[I]
print("for testing-set users not present in training set:")
for s, f in metrics.items():
    print("\t{}: {:f}".format(s, f(_cond, _pred)))

Note the (slight) difference in accuracy metrics between the two groups of users above.

## Practice

Try modifying the hyper-parameters (defined in the `hyper` dictionary in
[Section 5](#5-Train-model)). Re-train the model and see how the accuracy metrics are
affected.


# 7 Compare to Baseline

It might be informative to compare our recommender's performance to a
simple baseline; for example, we might consider merely recommending
items according to their popularity. We'll compute items' popularity
from the training set such that the popularity of an item $i$ is the
number of training-set interactions with $i$ divided by the total number
of training-set interactions.

In [ ]:
#| code-summary: "Compute popularity of items for simple baseline"

item_pop = np.ravel(
    data["tr_full"]["ui"].sum(axis=0) / data["tr_full"]["ui"].sum()
)
assert item_pop.shape[0] == len(item_map)

Next we'll use these proportions to make recommendations for the testing
set. This is easy: For each user-item pair $(u,i),$ the score of $(u,i)$
is merely the popularity of $i$.

In [ ]:
#| code-summary: "Use popularity to make recommendations for test data"
yhat = item_pop[data["te"]["ui"].col]

Now we can evaluate this simple algorithm like we've done for LightFM to
compare the two.

In [ ]:
#| code-summary: "Evaluate simple baseline on all users in test set"
cond = sp.coo_matrix(data["te"]["ui"])
pred = sp.coo_matrix((yhat, (cond.row, cond.col)), shape=cond.shape)
_cond = cond.tolil()
_pred = pred.tolil()
for s, f in metrics.items():
    print("{}: {:f}".format(s, f(_cond, _pred)))

Notice the difference in these accuracy metrics compared to the those associated
with the LightFM model. 



# References

Kula, Maciej. 2015. “Metadata Embeddings for User and Item Cold-Start Recommendations.” In Proceedings of the 2nd Workshop on New Trends on Content-Based Recommender Systems Co-Located with 9th ACM Conference on Recommender Systems (RecSys 2015), Vienna, Austria, September 16-20, 2015, edited by Toine Bogers and Marijn Koolen, 1448:14–21. CEUR-WS.org. http://ceur-ws.org/Vol-1448/paper4.pdf.

Wu, Fangzhao, Ying Qiao, Jiun-Hung Chen, Chuhan Wu, Tao Qi, Jianxun Lian, Danyang Liu, et al. 2020. “MIND: A Large-Scale Dataset for News Recommendation.” In, 3597–3606. Association for Computational Linguistics. https://doi.org/10.18653/v1/2020.acl-main.331.